In [2]:
import numpy as np

from contextlib import suppress
with suppress(Exception):from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical


In [3]:
# For reproducibility
np.random.seed(1000)

In [4]:
if __name__ == '__main__':
    # Load the dataset
    (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

    # Create the model
    model = Sequential()

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='selu', kernel_initializer='lecun_uniform'))
    model.add(Dense(512, activation='selu', kernel_initializer='lecun_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    opt = Adam(lr=0.0001, decay=2e-6)
    
    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    # Train the model
    model.fit(X_train, to_categorical(Y_train),
              batch_size=128,
              shuffle=True,
              epochs=250,
              validation_data=(X_test, to_categorical(Y_test)),
              callbacks=[EarlyStopping(min_delta=0.001, patience=3)])

    # Evaluate the model
    scores = model.evaluate(X_test, to_categorical(Y_test))

    print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

170500096/170498071 [==============================] - 16s 0us/step
Train on 50000 samples, validate on 10000 samples
Epoch 1/250
50000/50000 [==============================] - 46s 917us/step - loss: 2.9267 - acc: 0.2677 - val_loss: 1.5022 - val_acc: 0.4518
Epoch 2/250
50000/50000 [==============================] - 35s 708us/step - loss: 1.5973 - acc: 0.4158 - val_loss: 1.3711 - val_acc: 0.5074
Epoch 3/250
50000/50000 [==============================] - 36s 710us/step - loss: 1.4205 - acc: 0.4874 - val_loss: 1.1844 - val_acc: 0.5705
Epoch 4/250
50000/50000 [==============================] - 36s 711us/step - loss: 1.2968 - acc: 0.5338 - val_loss: 1.1551 - val_acc: 0.5944
Epoch 5/250
50000/50000 [==============================] - 36s 712us/step - loss: 1.2010 - acc: 0.5732 - val_loss: 1.0406 - val_acc: 0.6339
Epoch 6/250
50000/50000 [==============================] - 36s 711us/step - loss: 1.1174 - acc: 0.6044 - val_loss: 0.9980 - val_acc: 0.6509
Epoch 7/250
50000/50000 [=================

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)         147584    
__________

In [6]:
model.save_weights("CIFAR10.h5")